<a href="https://colab.research.google.com/github/dmswl0707/Face-Expression-Recognition_with-DNN/blob/main/training_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##torchvision.dataset
1. data_loader = torch.utils.data.DataLoader(imagenet_data,batch_size=4,shuffle=True,num_workers=a)
2. torchvision.datasets.데이터셋 명( root,  train = True , transform = None, target_transform = None, download = False)
- root: 저장할 디렉토리
- train: 다운받을 데이터셋 종류(train/val/test)
- download : true and false
- transform : 이미지 변형

In [ ]:

import os
from PIL import Image
import random
import numpy as np
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from .ckplus_res import CKPlusResDataset

from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

# ckplus_res 데이터 다운받고 불러오기
root = 'C:/Users/ChoiEunJi/Desktop/ckplus_res'

#불러온 데이터 로드하기

def load_data():
    transforms = transforms.Compose([transforms.ToTensor(), transforms.Nomalize((),())])

    train_dataset = torchvision.datasets.ckplus_res(root, train = True , download = True)
    test_dataset = torchvision.datasets.ckplus_res(root, test = True, download = True)

def split_train_val_test(train_dataset, test_dataset, p=0.8):
    train_set_num = int(len(train_dataset.data)*p)
    
    train_data, val_data = torch.utils.data.random_split(train_dataset, [train_set_num, len(train_dataset)-train_set_num])
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=True)
    test_loader=torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=True)
    
    return train_loader, val_loader, test_loader

#로드 할때 train 데이터에 20%는 validation을 위해 나



In [ ]:
#train 및 test하는 메소
def train(epoch, model, train_loader, val_loader):
    #model.train()

    train_loss = 0.0
    train_acc = 0.0
    global_step = 0
    for e in range(epoch):
        for batch_idx, (input, target) in enumerate(train_loader):
            global_step = e * len(train_loader) * batch_size + (batch_idx * batch_size)

            input, target = input.cuda(), target.cuda()
            input, target = Variable(input), Variable(target)

            optimizer.zero_grad()

            output = model(input)
            loss=F.nll_loss(output, target)
            preds = output.data.max(dim=1, keepdim=True)[1]
            acc=preds.eq(target.data.view_as(preds)).cpu().sum()
            train_loss +=F.nll_loss(output, target, reduction='sum').data.item()
            train_acc +=acc.item()

            loss.backward()
            optimizer.step()

        with torch.no_grad():
            val_acc, val_loss = test(model, val_loader)
            writer.add_scalar('Loss/val', val_loss, global_step)
            writer.add_scalar('Accuracy/val', val_acc, global_step)

        loss = train_loss/len(train_loader.dataset)
        accuracy = 100.0 * train_acc/len(train_loader.dataset)
        writer.add_scalar('Loss/train', loss, global_step)
        writer.add_scalar('Accuracy/train', accuracy, global_step)

        print('Epoch: {0:4} iter: {1:10} loss {2:3.5f} acc {3:3.5f} \t v_loss {4:3.5f} v_acc {5:3.5f}'
              .format(e+1, global_step, loss, accuracy, val_loss, val_acc))
        
        train_loss = 0.0
        train_acc = 0.0



In [ ]:
def test(model, data_loader):
    #model.eval()

    running_acc = 0.0
    running_loss = 0.0
    for batch_idx, (input, target) in enumerate(data_loader):

        input, target = input.cuda(), target.cuda()
        input, target = Variable(input), Variable(target)

        output = model(input)

        preds = output.data.max(dim=1, keepdim=True)[1]
        acc = preds.eq(target.data.view_as(preds)).cpu().sum()
        running_loss +=F.nll_loss(output, target, reduction='sum').data.item()
        running_acc += acc.item()

    accuracy = 100.0 * running_acc / len(data_loader.dataset)
    loss = running_loss / len(data_loader.dataset)

    return accuracy, loss



#summary writer

In [ ]:
if __name__=="__main__":
    writer = SummaryWriter()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    #hyperparameter
    lr = 0.003
    epoch = 50
    batch_size = 512
    train_val_p=0.8

    #데이터 셋 로드 및 train, val, test로 데이터 셋 나눔
    train_dataset, test_dataset = load_mnist()
    train_loader, val_loader, test_loader = split_train_val_test(train_dataset, test_dataset, p=train_val_p)

    #모델 불러오고, optimizer 설정
    model = Net().to(device)                            # 모델 불러오기
    optimizer = optim.SGD(model.parameters(), lr)      #optimizer 설정

    #layer 정보 프린트 하기
    print(summary(model, input_size=(1, 28, 28)))

    train(epoch, model, train_loader, val_loader)   #학습
    acc, _ = test(model, test_loader)               #테스트

    print('test acc {0:3.5f}' .format(acc))
    writer.close()